In [1]:
import pandas as pd

# Importing dataframes
#df_tags = pd.read_csv('../data/raw/ml-25m/tags.csv')
df_gscores = pd.read_csv('../data/raw/ml-25m/genome-scores.csv')
df_gtags = pd.read_csv('../data/raw/ml-25m/genome-tags.csv')
#df_links = pd.read_csv('../data/raw/ml-25m/links.csv')
df_movies = pd.read_csv('../data/raw/ml-25m/movies.csv')
df_ratings = pd.read_csv('../data/raw/ml-25m/ratings.csv')

In [2]:
# Create combined dataframe from the smaller dataframes and cleaning it
df = pd.merge(df_movies, df_gscores, how="left", on='movieId')
df = pd.merge(df, df_gtags, how="left", on='tagId')
df.drop(columns=['tagId'], inplace=True)
df = df.groupby(['movieId', 'title', 'genres']).agg({'relevance': list, 'tag': list})
df = pd.merge(df, df_ratings, on='movieId', how='left')
df.drop(columns=['timestamp'], inplace=True)
df.dropna(inplace=True)
df['userId'] = df['userId'].astype(int)


In [3]:
df.head()

,movieId,relevance,tag,userId,rating
0,1,"[0.0287499999999999, 0.0237499999999999, 0.062...","[007, 007 (series), 18th century, 1920s, 1930s...",2,3.5
1,1,"[0.0287499999999999, 0.0237499999999999, 0.062...","[007, 007 (series), 18th century, 1920s, 1930s...",3,4.0
2,1,"[0.0287499999999999, 0.0237499999999999, 0.062...","[007, 007 (series), 18th century, 1920s, 1930s...",4,3.0
3,1,"[0.0287499999999999, 0.0237499999999999, 0.062...","[007, 007 (series), 18th century, 1920s, 1930s...",5,4.0
4,1,"[0.0287499999999999, 0.0237499999999999, 0.062...","[007, 007 (series), 18th century, 1920s, 1930s...",8,4.0


In [4]:
# Function to keep only top 50 most relevant tags
def keep_top_n_relevance(row, n=50):
    combined = list(zip(row['relevance'], row['tag']))
    combined_sorted = sorted(combined, key=lambda x: x[0], reverse=True)[:n]
    row['relevance'], row['tag'] = zip(*combined_sorted)
    return row

# Apply function to DataFrame
df = df.apply(keep_top_n_relevance, axis=1)

KeyboardInterrupt: 